### Creation of the environment

In [1]:
import os
os.environ['USE_AUTH_EPHEM'] = '0'

In [2]:
%tensorflow_version 2.x
!pip3 install --upgrade pip
#!pip install -qU t5
!pip3 install git+https://github.com/google-research/text-to-text-transfer-transformer.git #extra_id_x support

import functools
import os
import time
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import tensorflow.compat.v1 as tf
import tensorflow_datasets as tfds

import t5

#Set the base dir(Google cloud bucket)
BASE_DIR = "gs://code-generation"

#C.B.: SEQ_LENGTH for number of tokens
SEQ_LENGTH = 512

if not BASE_DIR or BASE_DIR == "gs://":
  raise ValueError("You must enter a BASE_DIR.")
ON_CLOUD = True


if ON_CLOUD:
  import tensorflow_gcs_config
  from google.colab import auth
  # Set credentials for GCS reading/writing from Colab and TPU.
  TPU_TOPOLOGY = "2x2"
  try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection
    TPU_ADDRESS = tpu.get_master()
    print('Running on TPU:', TPU_ADDRESS)
  except ValueError:
    raise BaseException('ERROR: Not connected to a TPU runtime; please see the previous cell in this notebook for instructions!')
  auth.authenticate_user()
  tf.config.experimental_connect_to_host(TPU_ADDRESS)
  tensorflow_gcs_config.configure_gcs_from_colab_auth()

tf.disable_v2_behavior()

# Improve logging.
from contextlib import contextmanager
import logging as py_logging

if ON_CLOUD:
  tf.get_logger().propagate = False
  py_logging.root.setLevel('INFO')

@contextmanager
def tf_verbosity_level(level):
  og_level = tf.logging.get_verbosity()
  tf.logging.set_verbosity(level)
  yield
  tf.logging.set_verbosity(og_level)

     |████████████████████████████████| 2.1 MB 5.3 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
  Cloning https://github.com/google-research/text-to-text-transfer-transformer.git to /tmp/pip-req-build-1j6z2e9p
  Running command git clone --filter=blob:none --quiet https://github.com/google-research/text-to-text-transfer-transformer.git /tmp/pip-req-build-1j6z2e9p
  Resolved https://github.com/google-research/text-to-text-transfer-transformer.git to commit c070da4626d936bab4039b007a5202f039d55f0a
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 KB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.7/90.7 KB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 45.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 KB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

Instructions for updating:
non-resource variables are not supported in the long term


### Loading of tsv files
With this script you can load each tsv file for finetuning.
Please be sure that the path to all tsv files are correct

In [3]:
train_construct_length = 251271
test__construct_length = 37722
eval__construct_length = 36734

train_block_length = 101646
test__block_length = 12706
eval__block_length = 12705

train_token_length = 307779
test__token_length = 38480
eval__token_length = 38475

In [4]:
#Validation(train and test on the same dataset)

nq_tsv_path_construct = {
    "train":      BASE_DIR + '/T5_extension/ft_datasets/construct_train.tsv',
    "validation": BASE_DIR + '/T5_extension/ft_datasets/construct_test.tsv',
}

num_nq_examples_construct = dict(train=train_construct_length, validation=test__construct_length)

In [5]:
#Validation(train and test on the same dataset)

nq_tsv_path_block = {
    "train":      BASE_DIR + '/T5_extension/ft_datasets/block_train.tsv',
    "validation": BASE_DIR + '/T5_extension/ft_datasets/block_test.tsv',
}

num_nq_examples_block = dict(train=train_block_length, validation=test__block_length)

In [6]:
#Validation(train and test on the same dataset)

nq_tsv_path_token = {
    "train":      BASE_DIR + '/T5_extension/ft_datasets/token_train.tsv',
    "validation": BASE_DIR + '/T5_extension/ft_datasets/token_test.tsv',
}

num_nq_examples_token = dict(train=train_token_length, validation=test__token_length)

### Preprocess of the dataset
In this step we preprocess the dataset.  
You have to change the path to vocab files (*vocab_model_path* and *vocab_path*)
We're going to preprocess all the tsv file so that T5 can use them for finetuning.

In [7]:
from t5.data import postprocessors as t5_postprocessors
from t5.seqio import Feature,SentencePieceVocabulary


# # Set the path of sentencepiece model and vocab files
# # Must be the same used for the pre-trained phase
vocab_model_path = BASE_DIR + '/T5_extension/code.model'
vocab_path = BASE_DIR + '/T5_extension/code.vocab'


TaskRegistry = t5.data.TaskRegistry
TfdsTask = t5.data.TfdsTask


def get_default_vocabulary():
  return SentencePieceVocabulary(vocab_model_path, 100)

DEFAULT_OUTPUT_FEATURES = {
    "inputs": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True, required=False),

    "targets": Feature(
        vocabulary=get_default_vocabulary(), add_eos=True)
}

JAVA CONSTRUCT

In [8]:
def nq_construct(split, shuffle_files=True):
  # We only have one file for each split.
  del shuffle_files

   # Load lines from the text file as examples.

  ds = tf.data.TextLineDataset(nq_tsv_path_construct[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw train examples...")
for ex in tfds.as_numpy(nq_construct("train").take(5)):
  print(ex)

A few raw train examples...
{'input': b'private String formatEventDateRange(Date beginDate, Date endDate) { if ( <extra_id_0>) { if (isEndOfDay(endDate)) { return formatEventDate(beginDate); } else if (isMidnight(beginDate)) { return formatEventDate(beginDate) + " until " + eventOutDayOnlyDf.format(endDate); } else { return formatEventDate(beginDate) + " - " + eventOutTimeOnlyDf.format(endDate); } } else { return formatEventDate(beginDate) + " - " + formatEventDate(endDate); } } <SEP> /** Get the string representation of an event date range. */', 'output': b'DateUtils.isSameDay(beginDate, endDate)'}
{'input': b'private String formatEventDateRange(Date beginDate, Date endDate) { if (DateUtils.isSameDay(beginDate, endDate)) { if ( <extra_id_0>) { return formatEventDate(beginDate); } else if (isMidnight(beginDate)) { return formatEventDate(beginDate) + " until " + eventOutDayOnlyDf.format(endDate); } else { return formatEventDate(beginDate) + " - " + eventOutTimeOnlyDf.format(endDate); } 

In [9]:
def construct_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['CONSTRUCT:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [10]:
t5.data.TaskRegistry.remove('construct')
t5.data.TaskRegistry.add(
    "construct",
    dataset_fn=nq_construct,
    splits=["train", "validation"],
    text_preprocessor=[construct_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_construct
)

In [11]:
nq_task = t5.data.TaskRegistry.get("construct")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": SEQ_LENGTH, "targets": SEQ_LENGTH})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...
{'inputs_pretokenized': b'CONSTRUCT:public void onBindViewHolder(@NonNull Rule holder, int position) { holder.setLast(hasScroll && position == lastItem); holder.setValue(values.get( <extra_id_0>)); } <SEP> /** Put data into the rule at the given position. */', 'inputs': array([12094, 30813,    78,  2825,    65,    44,  9748,  1351,  2984,
       23610, 18588,  7096,  6221,     9,    99,   961,    91,    20,
        6221,    12,  2060,  2844,   451,  6120, 10795,   311,   961,
         126,   315,   847,  4767,  6221,    12,  2060,   275,   451,
       14655,    12,  1323,   451, 32099,    91,  4767,    21,    52,
       25390,  1302,     6, 23808,  5583,   261,   216,    11,  1368,
          68,    11,  1017,   961,    12,   651,    98,     1],
      dtype=int32), 'targets_pretokenized': b'position', 'targets': array([961,   1], dtype=int32)}
{'inputs_pretokenized': b'CONSTRUCT:private void prepForModal() { RelativeLayout blackOutLayer = (Relativ

JAVA TOKEN

In [12]:
def nq_token(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_token[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_token("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public static Event getInfo <extra_id_0> Result result = Caller.getInstance().call("event.getInfo", apiKey, "event", eventId); return ResponseBuilder.buildItem(result, Event.class); } <SEP> /** * Get the metadata for an event on Last.fm. Includes attendance and lineup information. * * @param eventId The numeric last.fm event id * @param apiKey A Last.fm API key. * @return Event metadata */', 'output': b'(String eventId, String apiKey) {'}
{'input': b'private <extra_id_0> if (mGoogleApiClient.isConnected()) { Plus.AccountApi.clearDefaultAccount(mGoogleApiClient); Plus.AccountApi.revokeAccessAndDisconnect(mGoogleApiClient) .setResultCallback(new ResultCallback<Status>() { @Override public void onResult(Status arg0) { Log.e(TAG, "User access revoked!"); mGoogleApiClient.connect(); updateUI(false); } }); } } <SEP> /** * Revoking access from google * */', 'output': b'void revokeGplusAccess() {'}
{'input': b'private void revokeGplusAccess() { if (mGoog

In [13]:
def token_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['TOKEN:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [14]:
t5.data.TaskRegistry.remove('token')
t5.data.TaskRegistry.add(
    "token",
    dataset_fn=nq_token,
    splits=["train", "validation"],
    text_preprocessor=[token_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_token
)

In [15]:
nq_task = t5.data.TaskRegistry.get("token")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": SEQ_LENGTH, "targets": SEQ_LENGTH})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...
{'inputs_pretokenized': b'TOKEN:public BlockDeviceObserver(Shell rootShell, PartitionListener listener) { super("/dev/block/", FileObserver.CREATE | FileObserver.DELETE); mVolumes = new HashMap<String, Volume>(); mListener = listener; mHandler = new Handler(Looper.getMainLooper <extra_id_0> mRootShell = rootShell; detectDevices(); } <SEP> /** * Creates a new block device observer. * Does not start observing until {@link #startWatching()} is called. * @param rootShell The shell to execute mount commands in. * @param listener A listener to receive block device events. * Calls are received on the main thread. */', 'inputs': array([18964,    78,  2825,  3901,  3790,  8655,   451, 13259,  1475,
       13259,     9,  7277,  1128,  2173,    91,    20,   436,   451,
          26,    98, 11361,    98,  9062,    98,    26,     9,   629,
        8655,    12,  8543,  1338,   629,  8655,    12, 13112,  4767,
         101,  7491,     8,    24,    39,  1410,  4

JAVA BLOCK

In [16]:
def nq_block(split, shuffle_files=False):
  # We only have one file for each split.
  del shuffle_files

  # Load lines from the text file as examples.
  ds = tf.data.TextLineDataset(nq_tsv_path_block[split])
  ds = ds.map(
      functools.partial(tf.io.decode_csv, record_defaults=["string","string"],
                        field_delim="\t", use_quote_delim=False),
      num_parallel_calls=tf.data.experimental.AUTOTUNE)
  
  ds = ds.map(lambda *ex: dict(zip(["input", "output"], ex)))
  return ds

print("A few raw valid examples...")
for ex in tfds.as_numpy(nq_block("validation").take(5)):
  print(ex)

A few raw valid examples...
{'input': b'public void removeItem(int position, boolean argIsSilent) { if (position < 0) <extra_id_0> int size = mInternalPlaylist.size(); if (size > position) { mInternalPlaylist.remove(position); } if (!argIsSilent) notifyPlaylistChanged(); } <SEP> /** * * @param position */', 'output': b'{ VendorCrashReporter.report("Playlist remove", "Position must be greater or equal to zero"); return; }'}
{'input': b'public void removeItem(int position, boolean argIsSilent) { if (position < 0) { VendorCrashReporter.report("Playlist remove", "Position must be greater or equal to zero"); return; } int size = mInternalPlaylist.size(); if (size > position) <extra_id_0> if (!argIsSilent) notifyPlaylistChanged(); } <SEP> /** * * @param position */', 'output': b'{ mInternalPlaylist.remove(position); }'}
{'input': b'public void removeItem(int position, boolean argIsSilent) { if (position < 0) { VendorCrashReporter.report("Playlist remove", "Position must be greater or equal t

In [17]:
def block_preprocessing(ds):
  
  def to_inputs_and_targets(ex):

        inputs = tf.strings.join(['BLOCK:' + ex['input']], separator=' ')
        class_label = tf.strings.join([ex['output']], separator=' ')
        return {'inputs': inputs, 'targets': class_label }
    
  return ds.map(to_inputs_and_targets, 
                num_parallel_calls=tf.data.experimental.AUTOTUNE)

In [18]:
t5.data.TaskRegistry.remove('block')
t5.data.TaskRegistry.add(
    "block",
    dataset_fn=nq_block,
    splits=["train", "validation"],
    text_preprocessor=[block_preprocessing],
    output_features = DEFAULT_OUTPUT_FEATURES,
    metric_fns=[t5.evaluation.metrics.accuracy],
    num_input_examples=num_nq_examples_block
)

In [19]:
nq_task = t5.data.TaskRegistry.get("block")
ds = nq_task.get_dataset(split="train", sequence_length={"inputs": SEQ_LENGTH, "targets": SEQ_LENGTH})
print("A few preprocessed training examples...")
for ex in tfds.as_numpy(ds.take(5)):
  print(ex)


A few preprocessed training examples...
{'inputs_pretokenized': b'BLOCK:public static void toastShortMessage(final Activity activity, final String message) { if (!TextUtils.isEmpty(message)) { activity.runOnUiThread(new Runnable() { @Override public void run() <extra_id_0> }); } } <SEP> /** * Displays a short toast message on the screen. * * @param activity The activity that short message is displayed on it. * @param message The message to be displayed. */', 'inputs': array([17090,    78,  2825,   190,    65,    16,  2077,  6022,   538,
         451,  8685, 10647,  2223,     9,    85,    53,   504,    91,
          20,    56,     4,   125,  1147,   821,    12,   660,  3579,
         451,  5619,  6275,    20,  2223,    12,  5842,  1033, 10859,
        3369,   451,  1973,  6854,  1927,    20,    55,  8203,    38,
          65,   506,  1927, 32099,    21,  4767,    21,    21,    52,
       25390,  1302,     6, 23808,   651, 10120,     8,    18,   725,
          16,  2077,   504,    44,   

### Finetuning
You can run the finetuning using the following cells.  
Please set the correct path of the variable *MODEL_DIR* (the path to save the pretrained model in), *PATH_GIN_FILE* (the gin file configuration for this finetuning) and *PRETRAINED_DIR* (the folder that contains the pretrained model).  
**Keep attention** to change the *pretrained_model_dir* in finetune step (if you are starting the finetuning from scratch you have to set the value *PRETRAINED_DIR*, if you are restarting the finetuning from a previous saved checkpoint you have to set the value *MODEL_DIR*)

In [20]:
def _rate_num_input_examples(task):
  if "train" in task.splits:
    return float(task.num_input_examples("train"))
  elif "validation" in task.splits:
    return float(task.num_input_examples("validation"))
  else:
    raise ValueError("Task %s does not have a train or validation split." % (task.name))


t5.data.MixtureRegistry.remove("all_tasks")
t5.data.MixtureRegistry.add(
    "all_tasks",
    ["construct", "token", "block"],
    default_rate=_rate_num_input_examples
     #default_rate=1.0
)

In [21]:
from mesh_tensorflow.transformer.learning_rate_schedules import slanted_triangular
# C.B.: Added import
import t5.models

MODEL_SIZE = "small" 

# Set the folder where the checkpoints and all the others information will be writed
MODEL_DIR = BASE_DIR + '/T5_extension/finetuning'

# Specify the pre-trained dir which must contain the pre-trained models, the operative_config.gin file and the checkpoint file as well
PRETRAINED_DIR = BASE_DIR + '/T5_extension/pretrained_model'


model_parallelism, train_batch_size, keep_checkpoint_max = {
    "small": (1, 256, 16000),
    "base": (2, 128, 8),
    "large": (8, 64, 4),
    "3B": (8, 16, 1),
    "11B": (8, 16, 1)}[MODEL_SIZE]

tf.io.gfile.makedirs(MODEL_DIR)


# C.B.: Change to 100 for last phase, default 5000
SAVE_CHECKPOINTS_STEPS = 5000


model = t5.models.MtfModel(
    model_dir=MODEL_DIR,
    tpu=TPU_ADDRESS,
    tpu_topology=TPU_TOPOLOGY,
    model_parallelism=model_parallelism,
    batch_size=train_batch_size,
    learning_rate_schedule = slanted_triangular,
    sequence_length={"inputs": SEQ_LENGTH, "targets": SEQ_LENGTH},
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS,
    keep_checkpoint_max=keep_checkpoint_max if ON_CLOUD else None,
    iterations_per_loop=100,
)

In [ ]:
PATH_GIN_FILE = BASE_DIR + '/T5_extension/configuration_file/operative_config.gin'
import gin

with gin.unlock_config():
    gin.parse_config_file(PATH_GIN_FILE)
    #RUN FINE-TUNING
    FINETUNE_STEPS = 400000
    model.finetune(
        mixture_or_task_name="all_tasks",
        # pretrained_model_dir=PRETRAINED_DIR,
        pretrained_model_dir=MODEL_DIR,
        finetune_steps=FINETUNE_STEPS
    )

INFO:root:system_path_file_exists:gs://code-generation/T5_extension/configuration_file/operative_config.gin
ERROR:root:Path not found: gs://code-generation/T5_extension/configuration_file/operative_config.gin
INFO:root:system_path_file_exists:gs://code-generation/T5_extension/finetuning/operative_config.gin
ERROR:root:Path not found: gs://code-generation/T5_extension/finetuning/operative_config.gin


INFO:tensorflow:Using config: {'_model_dir': 'gs://code-generation/T5_extension/finetuning', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 5000, '_save_checkpoints_secs': None, '_session_config': graph_options {
  rewrite_options {
    disable_meta_optimizer: true
  }
}
cluster_def {
  job {
    name: "worker"
    tasks {
      key: 0
      value: "10.24.154.106:8470"
    }
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({'worker': ['10.24.154.106:8470']}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': 'grpc://10.24.154.106:8470', '_evaluation_master': 'grpc://10.24.154.106: